In [16]:
#Importing necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [17]:
#Load the dataset

data_dir = "E:\cancer"

In [18]:
#Data augumentation using imagedatagenerator

from tensorflow.keras.preprocessing.image import ImageDataGenerator
data = ImageDataGenerator(validation_split = 0.2)

In [19]:
#Set images size and batch size

BATCH_SIZE = 16

X = Y = 224

In [20]:
#Training datset

train_ds = data.flow_from_directory(data_dir,
                                   class_mode = "categorical",
                                   target_size = (X,Y),
                                   color_mode = "rgb",
                                   batch_size = BATCH_SIZE,
                                   shuffle = False,
                                   subset = "training",
                                   seed = 42)

Found 8560 images belonging to 4 classes.


In [21]:
#Validation datset

validation_ds = data.flow_from_directory(data_dir,
                                   class_mode = "categorical",
                                   target_size = (X,Y),
                                   color_mode = "rgb",
                                   batch_size = BATCH_SIZE,
                                   shuffle = False,
                                   subset = "validation",
                                   seed = 42)

Found 2140 images belonging to 4 classes.


In [22]:
#Import necessary modules for training the model

from tensorflow.keras.applications.efficientnet import EfficientNetB7 as PretrainedModel, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Flatten, Dense, Input

In [25]:
#training the base model

base_model = PretrainedModel(input_shape = (X, Y,3),
                     #Loading weights pre-trained on imagenet
                     weights = 'imagenet',
                     #Do not include the imagenet classifier at the top
                     include_top = False)

#Freezing the base model
base_model.trainable = False

#Create new model on top

data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
])

# The base model contains batch-normalization layers. 
# we want to keep them in inference mode when we unfreeze the base model for fine tuning,
# so we make sure that the base model is running in inference mode here
#print(base_model.input)

inputs = keras.Input(shape=(224, 224, 3))
#inputs=data_augmentation(inputs)
#print(inputs)

x=base_model(inputs,training=False)

x = GlobalAveragePooling2D()(x)
x=Flatten()(x)

x=Dense(128,activation='relu')(x)
x=Dense(64,activation='relu')(x)
y=Dense(4,activation='softmax')(x)

model=Model(inputs,outputs=y)

model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb7 (Functional)  (None, 7, 7, 2560)       64097687  
                                                                 
 global_average_pooling2d_3   (None, 2560)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 flatten_3 (Flatten)         (None, 2560)              0         
                                                                 
 dense_9 (Dense)             (None, 128)               327808    
                                                                 
 dense_10 (Dense)            (None, 64)                8256      
                                                           

In [26]:
#Train the top layer

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=[keras.metrics.CategoricalAccuracy()],
)

model.fit(train_ds, epochs=1, validation_data=validation_ds)

535/535 [==============================] - 3835s 7s/step - loss: 0.1064 - categorical_accuracy: 0.9612 - val_loss: 0.0520 - val_categorical_accuracy: 0.9785


In [27]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb7 (Functional)  (None, 7, 7, 2560)       64097687  
                                                                 
 global_average_pooling2d_3   (None, 2560)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 flatten_3 (Flatten)         (None, 2560)              0         
                                                                 
 dense_9 (Dense)             (None, 128)               327808    
                                                                 
 dense_10 (Dense)            (None, 64)                8256      
                                                           

In [30]:
#Fine-Tuning the model

# Unfreeze the model.
# Note that it keeps running in inference mode since we passed training = False when calling it.
# This means that the batch-normalization layers will not update their batch statistics.
# This prevents the batch-normalization layers from undoing all the training that we have done so far

base_model.trainable = True
for layer in base_model.layers:
        layer.trainable = False
        if isinstance(layer, keras.layers.BatchNormalization):
            layer.trainable = True
            layer.momentum = 0.99
model.summary()




Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb7 (Functional)  (None, 7, 7, 2560)       64097687  
                                                                 
 global_average_pooling2d_3   (None, 2560)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 flatten_3 (Flatten)         (None, 2560)              0         
                                                                 
 dense_9 (Dense)             (None, 128)               327808    
                                                                 
 dense_10 (Dense)            (None, 64)                8256      
                                                           

In [1]:
model.compile(
    optimizer=keras.optimizers.Adam(1e-5),
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=[keras.metrics.CategoricalAccuracy()],
)

model.fit(train_ds, epochs=1, validation_data=validation_ds)



NameError: name 'model' is not defined